In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [16]:
# 데이터 불러오기

base_path = '../../../Machine-Running_LJW/Group Project/'

submission_path = base_path + 'sample_submission_V2.csv' 
train_path = base_path + 'train_V2.csv'
test_path = base_path + 'test_V2.csv'

submission = pd.read_csv(submission_path)
train= pd.read_csv(train_path)
test=pd.read_csv(test_path)

In [ ]:
# # Kaggle 제출 시

# # 데이터 불러오기

# submission_path = '/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv' 
# train_path = '/kaggle/input/pubg-finish-placement-prediction/train_V2.csv'
# test_path = '/kaggle/input/pubg-finish-placement-prediction/test_V2.csv'

# submission = pd.read_csv(submission_path)
# train= pd.read_csv(train_path)
# test=pd.read_csv(test_path)

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost
from sklearn.metrics import mean_absolute_error

In [18]:
# 데이터프레임 각각의 행과 열 개수 확인

test.shape,train.shape,submission.shape

((1934174, 28), (4446966, 29), (1934174, 2))

In [19]:
# 결측치 제거

train01 = train.dropna()

In [20]:
# train data에서 사용할 feature columns 정의

df_col = [ 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc']

In [21]:
# train data에서 위에서 정의한 'df_col' columns에 대해 'groupId' 기준으로 집계하여 평균값으로 채운다.

train02 = train01.groupby(['groupId'])[df_col].agg('mean').reset_index()

In [22]:
train02

,groupId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,00000c08b5be36,0.000000,0.333333,247.166667,1.666667,0.333333,0.333333,50.333333,0.0,0.666667,...,0.000000,0.0000,0.0,0.000000,0.0,0.0,271.533333,2.333333,0.0,0.2000
1,00000d1cbbc340,0.000000,0.000000,173.700000,0.000000,0.000000,0.000000,40.000000,0.0,1.000000,...,0.000000,0.0000,0.0,0.000000,0.0,0.0,135.800000,3.000000,0.0,0.1154
2,000025a09dd1d7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,72.000000,0.0,0.000000,...,0.000000,0.0000,0.0,0.000000,0.0,0.0,110.800000,2.000000,0.0,0.2000
3,000038ec4dff53,0.666667,2.333333,263.556667,1.666667,0.333333,3.000000,13.000000,0.0,2.000000,...,0.666667,0.0000,0.0,74.923333,0.0,0.0,2082.666667,5.666667,0.0,0.9615
4,00003a54230763,0.500000,0.000000,50.000000,0.500000,0.500000,0.000000,54.500000,1013.0,0.500000,...,0.000000,0.0000,0.0,0.000000,0.0,0.0,92.675000,2.500000,1501.0,0.1395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026739,ffffedb6ecff6a,0.000000,0.000000,50.000000,0.500000,0.000000,0.000000,87.500000,0.0,0.000000,...,0.000000,0.0000,0.0,0.000000,0.0,0.0,156.100000,1.500000,0.0,0.0370
2026740,fffff305a0133d,0.500000,2.500000,208.350000,1.000000,1.000000,2.500000,18.000000,0.0,2.000000,...,0.000000,82.2514,0.0,1.207000,0.0,0.0,2562.000000,4.000000,0.0,0.8511
2026741,fffff32bc7eab9,0.000000,0.000000,300.000000,0.000000,1.000000,0.000000,10.000000,1166.0,3.000000,...,0.000000,0.0000,0.0,0.000000,0.0,0.0,114.700000,1.000000,1443.0,0.2500
2026742,fffff7edfc4050,0.000000,3.000000,267.069000,1.000000,1.500000,1.500000,14.000000,0.0,2.500000,...,0.500000,5288.5000,0.0,0.000000,0.0,0.0,2283.500000,6.000000,0.0,0.9783


In [23]:
# test data에서 사용할 feature columns 정의

df_col = [ 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']

In [24]:
# test data에서 위에서 정의한 'df_col' columns에 대해 'groupId' 기준으로 집계하여 평균값으로 채운다.
# test01은 아래에서 scaling 후 모델링 하여 target data를 찾아올 용도로 사용 예정
test01 = test.groupby(['groupId'])[df_col].agg('mean').reset_index()

In [25]:
# test02는 추후 merge 용도로 쓰기 위해 아래에서 test01을 통해 학습한 데이터 Y를 test02의 'winPlacePerc' column에 붙인 후,
# test02에서 'groupId' 및 'winPlacePerc' columns를 추출하여 test04로 사용 예정
test02 = test01.copy()

In [26]:
# 사용하지 않는 columns 제거
train02 = train02.drop(columns=["killPlace","groupId","kills"])
test01 = test01.drop(columns=['killPlace','groupId','kills'])
test02 = test02.drop(columns=['killPlace','kills'])

In [27]:
# 결측치 제거 대상 columns

check_column = ['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
        'killStreaks', 'longestKill', 'maxPlace',
       'numGroups', 'revives', 'rideDistance', 'roadKills', 'swimDistance',
       'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired']#18

In [31]:
df_t_IQR = train02.copy()
for i in check_column:
    quantile_25 = np.percentile(train02[i].values, 25)
    quantile_75 = np.percentile(train02[i].values, 75)
    
    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*1.5
  
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
    outlier_idx = df_t_IQR[i][ (df_t_IQR[i] < lowest) | (df_t_IQR[i] > highest) ].index
    df_t_IQR = train02.drop(outlier_idx, axis=0)

In [32]:
x=df_t_IQR.drop(columns='winPlacePerc')
y=df_t_IQR[['winPlacePerc']]
scaler = StandardScaler()
x=scaler.fit_transform(x)
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=0)

In [33]:
test01sc=scaler.fit_transform(test01)
test01sc

array([[ 0.35496467,  3.12433226, -0.22425092, ...,  0.27456176,
         0.60464791, -0.81332517],
       [ 0.76407843, -0.70921158, -0.58581524, ..., -0.7632628 ,
         0.13165942, -0.81332517],
       [ 0.76407843,  0.19873302, -0.14519718, ...,  1.05868741,
         1.78711911,  1.21849276],
       ...,
       [ 0.76407843, -0.40656338, -0.22552708, ..., -0.84012351,
        -0.81431754, -0.81332517],
       [ 0.76407843, -0.70921158, -0.22562963, ..., -0.85864475,
        -0.10483482,  0.97889797],
       [ 0.1504078 , -0.70921158, -0.434043  , ..., -0.97535883,
        -1.40555314, -0.81332517]])

In [34]:
xgb_model = xgboost.XGBRegressor(
    eval_metric=mean_absolute_error
    ,random_state=0)

xgb_model.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=<function mean_absolute_error at 0x000002A802A16AF0>,
             gamma=0, gpu_id=-1, grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, reg_lambda=1, ...)

In [35]:
# 모델로 타겟값을 뽑아낸것
Y=xgb_model.predict(test01sc)

In [36]:
len(Y) 

886238

In [37]:
#제출용 y값을 붙이는 곳
len(test)

1934174

In [38]:
test02['winPlacePerc']=Y

In [39]:
test04 =test02.loc[:,['groupId','winPlacePerc']]#작은 테스트 -> 그룹아이디로 test에 붙여야 한다

In [40]:
test03 = test.loc[:,'groupId']

In [41]:
test03.shape,test04.shape

((1934174,), (886238, 2))

In [42]:
test02.isnull().sum()

groupId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPoints         0
killStreaks        0
longestKill        0
matchDuration      0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       0
dtype: int64

In [43]:
target = pd.merge(test03, test04, how='left', on='groupId')

In [44]:
len(target)

1934174

In [31]:
# Kaggle 제출 시 사용
# submission에 그 결과값을 담아서 제출하기
submission['winPlacePerc'] = target.winPlacePerc
submission.to_csv('submission.csv', index = False)